<a href="https://colab.research.google.com/github/debu-sinha/building-ai-agents/blob/main/Basic_ReAct_Agent.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
#Install all dependency packages
#Remember to execute this before running any of the exercises

!pip install langchain
!pip install langchain-openai
!pip install langchain_community
!pip install langchain_chroma
!pip install langgraph
!pip install tenacity
!pip install pysqlite3-binary
!pip install pandas
!pip install pypdf
!pip install nbformat

### Setup Function Tools for ReAct Agent

In [3]:
from langchain_core.tools import tool

@tool
def find_sum(x: int, y:int) -> int :
  """
  This function takes as input two integers and returns their sum
  """
  return x+y

@tool
def find_product(x: int, y: int) -> int:
  """
  This function takes as input two integers and returns their product.
  """
  return x*y

### Create a basic ReAct Agent

In [6]:
from google.colab import userdata
from langchain_openai import ChatOpenAI

model = ChatOpenAI(model_name="gpt-3.5-turbo", openai_api_key=userdata.get('oai'))

response = model.invoke("Hello, how are you?")
print(response.content)

Hello! I'm just a computer program, so I don't have feelings, but I'm here and ready to assist you with anything you need. How can I help you today?


In [8]:
from langgraph.prebuilt import create_react_agent
from langchain_core.messages import SystemMessage

agent_tools = [find_sum, find_product]

#System prompt
system_prompt=SystemMessage(
    """You are a Math genius who can solve math problems. Solve the
    problems provided by the user, by using only tools available.
    Do not solve the problem yourself"""
)

agent_graph=create_react_agent(
    model,
    prompt=system_prompt,
    tools=agent_tools)

### Execute the ReAct Agent

In [10]:
#Example 1
inputs = {"messages":[("user","what is the sum of 2 and 3 ?"), ("user","what is the product of 2 and 3 ?")]}

result = agent_graph.invoke(inputs)

#Get the final answer
print(f"Agent returned : {result['messages'][-1].content} \n")

print("Step by Step execution : ")
for message in result['messages']:
    print(message.pretty_repr())


Agent returned : The sum of 2 and 3 is 5, and the product of 2 and 3 is 6. 

Step by Step execution : 
================================ Human Message =================================

what is the sum of 2 and 3 ?
================================ Human Message =================================

what is the product of 2 and 3 ?
================================== Ai Message ==================================
Tool Calls:
  find_sum (call_aTRZQg0yWyznpB4bmExl36fB)
 Call ID: call_aTRZQg0yWyznpB4bmExl36fB
  Args:
    x: 2
    y: 3
  find_product (call_IF9OqaC60gPs1gNLCEYf4IZl)
 Call ID: call_IF9OqaC60gPs1gNLCEYf4IZl
  Args:
    x: 2
    y: 3
================================= Tool Message =================================
Name: find_sum

5
================================= Tool Message =================================
Name: find_product

6
================================== Ai Message ==================================

The sum of 2 and 3 is 5, and the product of 2 and 3 is 6.


### Debugging the Agent

In [11]:
agent_graph=create_react_agent(
    model,
    prompt=system_prompt,
    tools=agent_tools,
    debug=True)

inputs = {"messages":[("user","what is the sum of 2 and 3 ?")]}

result = agent_graph.invoke(inputs)

[values] {'messages': [HumanMessage(content='what is the sum of 2 and 3 ?', additional_kwargs={}, response_metadata={}, id='754e283f-79ab-4479-840a-385eac716b37')]}
[updates] {'agent': {'messages': [AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_Yet8PvvfKbofEafsDeYWC7Hc', 'function': {'arguments': '{"x":2,"y":3}', 'name': 'find_sum'}, 'type': 'function'}], 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 18, 'prompt_tokens': 132, 'total_tokens': 150, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'id': 'chatcmpl-CDAVvFuHN3Heu85565JGMY3avlvE8', 'service_tier': 'default', 'finish_reason': 'tool_calls', 'logprobs': None}, id='run--02a7bbb0-4e4a-417a-83c5-bfa207331dd6-0', tool_calls=[{'name': 'find_sum', 'args': {'x': 2, 'y'

# Task
Improve the existing agent to handle multiple users, maintain separate conversation histories for each user, and simulate user-specific data access, providing a step-by-step implementation.

## Implement a basic user identification system

### Subtask:
We'll start by adding a simple way to identify users in our current notebook environment.


**Reasoning**:
Add a markdown cell to explain the purpose of the `current_user_id` variable.



In [12]:
current_user_id = "user_123"

### User Identification Simulation

For this demonstration, we are using the `current_user_id` variable to simulate a unique identifier for a logged-in user. In a real-world application, this ID would typically come from an authentication system.

## Set up in-memory conversation storage

### Subtask:
We'll use a Python dictionary to store conversation history in memory for different users. This will be a temporary solution to demonstrate the concept before moving to a persistent storage.


**Reasoning**:
Initialize an empty dictionary to store conversation history for different users and print it to verify it's empty.



In [14]:
conversation_history = {}
print(conversation_history)

{}


## Modify the agent to use conversation history

### Subtask:
We'll adapt the agent's invocation to take and return conversation history, allowing it to maintain context for each user.


**Reasoning**:
Define the function `handle_user_query` to manage conversation history for each user and interact with the agent graph.



In [16]:
from langchain_core.messages import HumanMessage, AIMessage

def handle_user_query(agent_graph, user_query: str, user_id: str):
    """
    Handles user queries, maintaining conversation history for each user.

    Args:
        agent_graph: The LangChain agent graph.
        user_query: The current user query string.
        user_id: The unique identifier for the user.

    Returns:
        The agent's response as a string.
    """
    # Retrieve or initialize conversation history for the user
    if user_id not in conversation_history:
        conversation_history[user_id] = []

    # Append the current user query
    conversation_history[user_id].append(HumanMessage(content=user_query))

    # Invoke the agent graph with the user's history
    result = agent_graph.invoke({"messages": conversation_history[user_id]})

    # Extract the agent's response
    agent_response = result['messages'][-1].content

    # Append the agent's response to the history
    conversation_history[user_id].append(AIMessage(content=agent_response))

    return agent_response


In [21]:
conversation_history

{'user_Charlie': [HumanMessage(content='what is the sum of 10 and 15?', additional_kwargs={}, response_metadata={}, id='6b6b17ee-7867-4de4-ae52-9d110ffda4e4'),
  AIMessage(content='The sum of 10 and 15 is 25.', additional_kwargs={}, response_metadata={}, id='ef0e645f-5c1d-4cae-bdc1-261e7573834e'),
  HumanMessage(content='what is the product of 4 and 7?', additional_kwargs={}, response_metadata={}, id='5ae0012f-d200-45fb-a00a-9c1751a6de61'),
  AIMessage(content='The sum of 4 and 7 is 11, and the product of 4 and 7 is 28.', additional_kwargs={}, response_metadata={})],
 'user_Alice': [HumanMessage(content='what is the sum of 50 and 100?', additional_kwargs={}, response_metadata={}, id='e2730c5e-6561-4aa9-aa14-fbdc0d8208d8'),
  AIMessage(content='The sum of 50 and 100 is 150.', additional_kwargs={}, response_metadata={}, id='6e3629a8-6fe7-402c-b840-86411c62a670'),
  HumanMessage(content='what is the product of 9 and 9?', additional_kwargs={}, response_metadata={}, id='1bac7c21-f4d2-4e2a-b

## Simulate multiple users

### Subtask:
Simulate interactions from different users to test if the agent correctly handles separate conversations.


**Reasoning**:
Simulate interactions from different users by iterating through a list of queries, randomly selecting a user for each query, invoking the handle_user_query function, and printing the interaction details. After the simulation, print the conversation_history to verify that separate histories are maintained.



In [17]:
import random
from langchain_core.messages import HumanMessage, AIMessage

user_ids = ["user_Alice", "user_Bob", "user_Charlie"]
queries = [
    "what is the sum of 10 and 15?",
    "what is the product of 4 and 7?",
    "what is the sum of 50 and 100?",
    "what is the product of 9 and 9?"
]

# Ensure the agent_graph object exists from previous steps
if 'agent_graph' not in globals():
    print("Error: agent_graph is not defined. Please run the previous cells.")
else:
    for query in queries:
        user_id = random.choice(user_ids)
        print(f"--- User: {user_id}, Query: {query} ---")
        response = handle_user_query(agent_graph, query, user_id)
        print(f"Agent Response: {response}\n")

    print("\n--- Final Conversation History ---")
    for user_id, history in conversation_history.items():
        print(f"History for {user_id}:")
        for message in history:
            print(f"  {message.type}: {message.content}")
        print("-" * 20)


--- User: user_Charlie, Query: what is the sum of 10 and 15? ---
[values] {'messages': [HumanMessage(content='what is the sum of 10 and 15?', additional_kwargs={}, response_metadata={}, id='6b6b17ee-7867-4de4-ae52-9d110ffda4e4')]}
[updates] {'agent': {'messages': [AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_TBzYtOThRCNVb5GpJbRL5kD2', 'function': {'arguments': '{"x":10,"y":15}', 'name': 'find_sum'}, 'type': 'function'}], 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 18, 'prompt_tokens': 132, 'total_tokens': 150, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'id': 'chatcmpl-CDCYFw3oL3oQAZaHNd9K7TBB0Amxy', 'service_tier': 'default', 'finish_reason': 'tool_calls', 'logprobs': None}, id='run--3dbe484b-4f76-4dd7-be51-44

## Introduce a basic data access simulation (optional)

### Subtask:
To simulate data access control, we can create a simple dictionary representing a vector store with data associated with different user IDs.


**Reasoning**:
Create and populate the user_data_store dictionary and then print it to verify its contents.



In [18]:
user_data_store = {
    "user_Alice": ["Alice's document 1", "Alice's document 2"],
    "user_Charlie": ["Charlie's report"],
    "user_Bob": ["Bob's spreadsheet", "Bob's presentation"]
}
print(user_data_store)

{'user_Alice': ["Alice's document 1", "Alice's document 2"], 'user_Charlie': ["Charlie's report"], 'user_Bob': ["Bob's spreadsheet", "Bob's presentation"]}


## Modify the agent to use simulated data access (optional)

### Subtask:
Modify the agent to use simulated data access (optional)


**Reasoning**:
Define a new tool function `retrieve_user_data` to access the simulated user data store, and then add this tool to the agent's tool list.



In [19]:
from langchain_core.tools import tool

@tool
def retrieve_user_data(user_id: str) -> list:
  """
  Retrieves a list of documents accessible by the given user ID.
  """
  return user_data_store.get(user_id, ["No data found for this user."])

# Add the new tool to the agent tools
agent_tools.append(retrieve_user_data)

# Recreate the agent graph with the updated tool list
agent_graph = create_react_agent(
    model,
    prompt=system_prompt,
    tools=agent_tools
)

**Reasoning**:
Update the system prompt to inform the agent about the new tool and instruct it to use it when necessary, then finish the task.



In [20]:
from langchain_core.messages import SystemMessage

# Update the system prompt
system_prompt = SystemMessage(
    """You are a Math genius who can solve math problems and also retrieve user-specific data.
    Solve the math problems provided by the user, by using only tools available.
    If a user asks about their data or documents, use the 'retrieve_user_data' tool with their user ID.
    Do not solve the problem yourself or access data directly."""
)

# Recreate the agent graph with the updated system prompt
agent_graph = create_react_agent(
    model,
    prompt=system_prompt,
    tools=agent_tools
)

## Summary:

### Data Analysis Key Findings

*   A basic user identification system was implemented by creating a `current_user_id` variable, although the attempt to add a markdown explanation initially failed due to using the incorrect command.
*   An in-memory Python dictionary named `conversation_history` was successfully initialized to store conversation history for different users, with each user ID mapping to their unique list of messages.
*   A function `handle_user_query` was created to manage user-specific conversation history, append new messages, invoke the agent with the relevant history, and update the history with the agent's response.
*   Simulation with multiple users demonstrated that the agent successfully handled separate conversations and maintained distinct conversation histories for each user in the `conversation_history` dictionary.
*   A dictionary `user_data_store` was successfully created and populated to simulate user-specific data access.
*   A new tool function `retrieve_user_data` was defined and added to the agent's toolset to simulate retrieving user-specific data from the `user_data_store`.
*   The agent's system prompt was updated to instruct the agent to use the `retrieve_user_data` tool when a user asks about their data, ensuring the agent utilizes the simulated data access mechanism.

### Insights or Next Steps

*   Integrate a more robust user authentication system and persistent storage for conversation history (e.g., database) for a production environment.
*   Further test the agent's ability to use the `retrieve_user_data` tool in simulated conversations with different users, including cases where a user's data is not found.
